<a href="https://colab.research.google.com/github/Luicazen/NLPTweets/blob/Rf/initRF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
!pip install category_encoders

In [46]:
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

In [2]:
from google.colab import files
uploaded = files.upload()
import pandas as pd
import numpy as np

Saving test.csv to test (1).csv
Saving train.csv to train (1).csv


In [23]:
#Read in Training data
trainingDF = pd.read_csv('train.csv')
trainingDF.index +=1
print("Training Data Summary")
display(trainingDF.shape)
display(trainingDF.describe())
display(trainingDF.head())
trainingDF.info()
print(trainingDF.columns)

Training Data Summary


(7613, 5)

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


,id,keyword,location,text,target
1,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
2,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
3,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
4,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
5,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 1 to 7613
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB
Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')


In [27]:
#Check for null data
print("Training Data")
trainingDF.isnull().sum()

Training Data


,0
id,0
keyword,61
location,2533
text,0
target,0


In [33]:
X = trainingDF.drop(['target'], axis=1)
y = trainingDF['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)
X_train.shape, X_test.shape
X_train.dtypes
X_train.head()


,id,keyword,location,text
5837,8339,rubble,"Dallas, Tejas",Photo: postapocalypticflimflam: Prodding aroun...
31,44,NaN,NaN,The end!
1880,2700,crush,NaN,Man crush everyday ???? @CristianInspire http...
6853,9820,trauma,I rap to burn shame.,@PTSD_Chat Yes. I feel the root of that is Sha...
2674,3835,detonate,"Sharkatraz/Bindle's Cleft, PA",@AutoAmes everyone hoped we would join ISIS an...


In [40]:
encoder = ce.OrdinalEncoder(cols=['id', 'keyword', 'location', 'text'])

X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

In [42]:
X_train.head()
X_test.head()

,id,keyword,location,text
2645,-1.0,121,2374.0,-1.0
2228,-1.0,55,2374.0,-1.0
5449,-1.0,129,226.0,-1.0
133,-1.0,18,2374.0,-1.0
6846,-1.0,3,-1.0,-1.0


In [48]:
rfc = RandomForestClassifier(random_state=0)
# fit the model
rfc.fit(X_train, y_train)
# Predict the Test set results
y_pred = rfc.predict(X_test)
# Check F1 score

print('Model F1 score with 10 decision-trees : {0:0.4f}'. format(f1_score(y_test, y_pred)))

Model F1 score with 10 decision-trees : 0.2062


# Task
Improve the F1 score of the current RandomForestClassifier model by analyzing its performance, implementing advanced text preprocessing (TF-IDF vectorization) for the 'text' column, tuning hyperparameters for the RandomForestClassifier, exploring alternative classification models, re-evaluating the best performing model, and finally summarizing the improvements and insights gained.

## Analyze Current Model Performance

### Subtask:
Generate a classification report and confusion matrix for the current RandomForestClassifier to understand its performance in more detail, identifying areas for improvement such as false positives and false negatives. This will help us pinpoint which classes are problematic.


**Reasoning**:
To analyze the current model performance in detail, I will import the necessary metrics functions from `sklearn.metrics` and then generate and print both the classification report and the confusion matrix using the predicted and true labels.



In [50]:
from sklearn.metrics import classification_report, confusion_matrix

# Generate classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Generate confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.96      0.74      1446
           1       0.70      0.12      0.21      1067

    accuracy                           0.60      2513
   macro avg       0.65      0.54      0.47      2513
weighted avg       0.64      0.60      0.51      2513


Confusion Matrix:
[[1391   55]
 [ 938  129]]


## Feature Engineering - Text Preprocessing

### Subtask:
Implement advanced text preprocessing using TF-IDF vectorization for the 'text' column and combine it with other ordinally encoded features.


**Reasoning**:
The first step is to re-split the original `trainingDF` into training and testing sets to ensure that the `text` column is available in its raw form for TF-IDF vectorization. This will create `X_train_raw`, `X_test_raw`, `y_train`, and `y_test`.



In [51]:
X_train_raw, X_test_raw, y_train, y_test = train_test_split(trainingDF.drop('target', axis=1), trainingDF['target'], test_size=0.33, random_state=42)

print("Shape of X_train_raw:", X_train_raw.shape)
print("Shape of X_test_raw:", X_test_raw.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

X_train_raw.head()

Shape of X_train_raw: (5100, 4)
Shape of X_test_raw: (2513, 4)
Shape of y_train: (5100,)
Shape of y_test: (2513,)


,id,keyword,location,text
5837,8339,rubble,"Dallas, Tejas",Photo: postapocalypticflimflam: Prodding aroun...
31,44,NaN,NaN,The end!
1880,2700,crush,NaN,Man crush everyday ???? @CristianInspire http...
6853,9820,trauma,I rap to burn shame.,@PTSD_Chat Yes. I feel the root of that is Sha...
2674,3835,detonate,"Sharkatraz/Bindle's Cleft, PA",@AutoAmes everyone hoped we would join ISIS an...


**Reasoning**:
Now that the raw data is split, I will initialize an `OrdinalEncoder` to encode the 'id', 'keyword', and 'location' columns in `X_train_raw` and `X_test_raw`.



In [52]:
encoder_non_text = ce.OrdinalEncoder(cols=['id', 'keyword', 'location'])

X_train_encoded_non_text = encoder_non_text.fit_transform(X_train_raw[['id', 'keyword', 'location']])
X_test_encoded_non_text = encoder_non_text.transform(X_test_raw[['id', 'keyword', 'location']])

print("Shape of X_train_encoded_non_text:", X_train_encoded_non_text.shape)
print("Shape of X_test_encoded_non_text:", X_test_encoded_non_text.shape)

X_train_encoded_non_text.head()

Shape of X_train_encoded_non_text: (5100, 3)
Shape of X_test_encoded_non_text: (2513, 3)


,id,keyword,location
5837,1,1,1
31,2,222,2374
1880,3,2,2374
6853,4,3,2
2674,5,4,3


**Reasoning**:
Now I will import `TfidfVectorizer` and then initialize it to transform the 'text' column of the raw training and testing data into TF-IDF features.



In [53]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_features=5000) # Using a max_features for better manageability

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_raw['text'].fillna(''))
X_test_tfidf = tfidf_vectorizer.transform(X_test_raw['text'].fillna(''))

print("Shape of X_train_tfidf:", X_train_tfidf.shape)
print("Shape of X_test_tfidf:", X_test_tfidf.shape)

Shape of X_train_tfidf: (5100, 5000)
Shape of X_test_tfidf: (2513, 5000)


**Reasoning**:
Now I will convert the encoded non-text features to sparse matrices and then horizontally stack them with the TF-IDF features to create the final processed training and testing datasets. This ensures all features are in a compatible format for model training.



In [54]:
from scipy.sparse import hstack, csr_matrix

# Convert encoded non-text DataFrames to sparse matrices
X_train_encoded_non_text_sparse = csr_matrix(X_train_encoded_non_text.values)
X_test_encoded_non_text_sparse = csr_matrix(X_test_encoded_non_text.values)

# Horizontally stack the non-text sparse features with the TF-IDF features
X_train_processed = hstack([X_train_encoded_non_text_sparse, X_train_tfidf])
X_test_processed = hstack([X_test_encoded_non_text_sparse, X_test_tfidf])

print("Shape of X_train_processed:", X_train_processed.shape)
print("Shape of X_test_processed:", X_test_processed.shape)

Shape of X_train_processed: (5100, 5003)
Shape of X_test_processed: (2513, 5003)


## Hyperparameter Tuning for RandomForestClassifier

### Subtask:
Optimize the hyperparameters of the RandomForestClassifier using techniques like GridSearchCV or RandomizedSearchCV. This will involve experimenting with parameters such as `n_estimators`, `max_depth`, `min_samples_split`, and `min_samples_leaf` to find the best configuration that maximizes the F1 score.


**Reasoning**:
To optimize the hyperparameters of the RandomForestClassifier, I will import GridSearchCV and define a parameter grid for tuning. Then, I will initialize a RandomForestClassifier, set up GridSearchCV with f1-scoring, fit it to the processed training data, and finally print the best parameters and corresponding F1 score.



In [55]:
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for RandomForestClassifier
param_grid = {
    'n_estimators': [100, 200, 300], # Number of trees in the forest
    'max_depth': [10, 20, 30, None], # Maximum number of levels in tree
    'min_samples_split': [2, 5, 10], # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4] # Minimum number of samples required to be at a leaf node
}

# Initialize a RandomForestClassifier model
rfc_model = RandomForestClassifier(random_state=0)

# Instantiate GridSearchCV
grid_search = GridSearchCV(estimator=rfc_model, param_grid=param_grid,
                           scoring='f1', cv=3, n_jobs=-1, verbose=2)

# Fit GridSearchCV to the processed training data
# Note: This step might take a significant amount of time
grid_search.fit(X_train_processed, y_train)

# Print the best parameters found by GridSearchCV
print("Best parameters found: ", grid_search.best_params_)

# Print the best F1 score achieved during the grid search
print("Best F1 score: ", grid_search.best_score_)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


KeyboardInterrupt: 